<a href="https://colab.research.google.com/github/priyankajuttu/FUTURE_ML_01/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip archive.zip


In [ ]:
import pandas as pd

df = pd.read_csv('Sample - Superstore.csv', encoding='ISO-8859-1')
print(df.head())
print(df.info())


In [ ]:
print(df.columns)
print(df.isnull().sum())
print(df.head())


In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
print(df['Order Date'].head())


In [ ]:
print(df.isnull().sum())


In [ ]:
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Day'] = df['Order Date'].dt.day
df['Weekday'] = df['Order Date'].dt.day_name()
print(df[['Order Date', 'Year', 'Month', 'Day', 'Weekday']].head())


In [ ]:
print(df.columns)


In [ ]:
# Replace 'Sales' with your actual sales amount column name
monthly_sales = df.groupby([df['Order Date'].dt.to_period('M')])['Sales'].sum()
print(monthly_sales.head())


In [ ]:
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Day'] = df['Order Date'].dt.day
df['Weekday'] = df['Order Date'].dt.day_name()

print(df[['Order Date', 'Year', 'Month', 'Day', 'Weekday']].head())


In [ ]:
# Aggregate monthly sales
monthly_sales = df.groupby(df['Order Date'].dt.to_period('M')).agg({'Sales': 'sum'})
monthly_sales = monthly_sales.reset_index()
monthly_sales['Order Date'] = monthly_sales['Order Date'].dt.to_timestamp()

print(monthly_sales.head())


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(monthly_sales['Order Date'], monthly_sales['Sales'], marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
!pip install prophet


In [ ]:
df_prophet = monthly_sales.rename(columns={'Order Date': 'ds', 'Sales': 'y'})
print(df_prophet.head())


In [ ]:
from prophet import Prophet

model = Prophet()
model.fit(df_prophet)  # Ensure df_prophet is already defined as shown earlier


In [ ]:
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)


In [ ]:
fig = model.plot(forecast)


In [ ]:
# Show last 12 months (which are the forecasted months)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12))


In [ ]:
fig2 = model.plot_components(forecast)


In [ ]:
# Add month column
monthly_sales['Month'] = monthly_sales['Order Date'].dt.month

# Mark potential holiday seasons (Nov-Dec; edit as per business knowledge)
monthly_sales['Holiday_Season'] = monthly_sales['Month'].isin([11, 12]).astype(int)
print(monthly_sales.head())


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

# Cross-validate: initial=80% of data, period=1 month, horizon=6 months
cv_results = cross_validation(model, initial='1000 days', period='30 days', horizon = '180 days')
perf = performance_metrics(cv_results)
print(perf[['horizon','mae','rmse','mape']].head())


In [ ]:
# Export files
monthly_sales.to_csv('monthly_sales.csv', index=False)
print("monthly_sales.csv created")

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].to_csv('monthly_forecast.csv', index=False)
print("monthly_forecast.csv created")

# List files to confirm
import os
print(os.listdir())


In [ ]:
!unzip archive.zip

In [ ]:
import pandas as pd

# Try with latin1 encoding
df = pd.read_csv('Sample - Superstore.csv', encoding='latin1')

# Aggregate sales by category
category_sales = df.groupby('Category')['Sales'].sum().reset_index()
category_sales = category_sales.sort_values('Sales', ascending=False)

print(category_sales)

# Export for Power BI
category_sales.to_csv('category_sales.csv', index=False)
